In [4]:
"""
MOVE BACK ONE DIRECTORY BEFORE RUNNING
REPOPULATES TMP FOLDER
TAKES A LONG TIME
"""

import os, re
import json
import math
import numpy as np
import pandas as pd

from utils.dataset import dataset

from tqdm import tqdm


SyntaxError: invalid syntax (<ipython-input-4-9deec044b0d7>, line 2)

In [2]:
description, df = dataset()

FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [16]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from tqdm import tqdm 
# declare an empty list to store
# latitude and longitude of values 
# of city column
longitude = []
latitude = []


# function to find the coordinate
# of a given city 
def findGeocode(city):
       
    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
    try:
          
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="your_app_name")
          
        return geolocator.geocode(city, timeout=10)
      
    except GeocoderTimedOut:
          
        return findGeocode(city)    

for round_ in range(int(df[df['TotalFatalInjuries']>0].shape[0]/500)):
    for i in tqdm(df[df['TotalFatalInjuries']>0]["Location"].iloc[round_*500:(round_+1)*500]):
        try:
            if findGeocode(i) != None:
                loc = findGeocode(i)
                latitude.append(loc.latitude)
                longitude.append(loc.longitude)

            else:
                latitude.append(np.nan)
                longitude.append(np.nan)
        except:
            latitude.append(np.nan)
            longitude.append(np.nan)
            continue
pd.DataFrame({'lat':latitude, 'long':longitude}).to_csv('tmp/latlongs_{}.csv'.format(round_))

100%|██████████| 500/500 [08:10<00:00,  1.02it/s]


In [9]:
import requests
import json

def get_city_opendata(city, country):
    tmp = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&q=%s&sort=population&facet=country&refine.country=%s'
    cmd = tmp % (city, country)
    res = requests.get(cmd)
    dct = json.loads(res.content)
    out = dct['records'][0]['fields']
    return out

def write_query(df_, i):
    df = df_.copy()
    
    populations = []
    geopoints = []
    try:
        country = pycountry.countries.search_fuzzy(df['Country'].iloc[i].lower())[0].alpha_2.lower()
        if country == 'us':
            tmp = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&q=%s&sort=population&facet=state&refine.state=%s'
            city = df["Location"].iloc[i].split(', ')[0].lower()
            state = df["Location"].iloc[i].split(', ')[1].lower()
            cmd = tmp % (city, state)
            res = requests.get(cmd)
            dct = json.loads(res.content)
            populations.append(dct['records'][0]['fields']['population'])
            geopoints.append(dct['records'][0]['fields']['geopoint'])
        else:
            tmp = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&q=%s&sort=population&facet=country&refine.country=%s'
            city = df["Location"].iloc[i].split(', ')[0].lower()
            cmd = tmp % (city, country)
            res = requests.get(cmd)
            dct = json.loads(res.content)
            populations.append(np.nan)
            geopoints.append(np.nan)
    except:
        populations.append(np.nan)
        geopoints.append(np.nan)

    with open('tmp/populations.txt', 'a') as writer:
        writer.write('%s\n' % str(populations[-1]))
    writer.close()
    
    with open('tmp/geopoints.txt', 'a') as writer:
        try:
            writer.write('%s %s\n' % (str(geopoints[-1][0]), str(geopoints[-1][1])))
        except:
            writer.write('missing\n')
    writer.close()
    
    return None


In [10]:
import pycountry


fatal_only = df[df['TotalFatalInjuries']>0]

rounds = int(fatal_only.shape[0]/500)

for round_ in range(rounds):
    for i in tqdm(range(round_*500,(round_+1)*500)):
        write_query(df_ = fatal_only, i = i)

100%|██████████| 500/500 [06:35<00:00,  1.26it/s]
